In [1]:
from stl_samplers import AugmentedOUDFollmerSDESTL
from config import get_architecture_specs, get_ref_process
from objectives import *
from toy_targets import funnel, carrillo
import matplotlib.pyplot as plt


In [2]:
architecture_specs = get_architecture_specs()
ref_process = get_ref_process()

model_def = AugmentedOUDFollmerSDESTL(
    sigma=ref_process.sigma,
    dim=ref_process.data_dim,
    drift_network=ref_process.drift_network,
    tfinal=ref_process.tfinal,
    dt=ref_process.dt,
    step_scheme=ref_process.step_scheme,
    alpha=ref_process.alpha,
    target=ref_process.target,
    tpu=ref_process.tpu,
    detach_stl_drift=ref_process.detach_stl_drift,
    diff_net=ref_process.diff_net,
    detach_dritf_path=ref_process.detach_dritf_path,
    detach_dif_path=ref_process.detach_dif_path,
    m=ref_process.m,
    log=ref_process.log,
    exp_bool=ref_process.exp_bool,
    exact=ref_process.exact
)

In [3]:
import time

all_trajectories = []

for i in range(1, 51):
    s_t = time.time()

    aug_trajectory = model_def(batch_size=200, is_training=True, ode=False)

    #print(aug_trajectory)
    all_trajectories.append(aug_trajectory)

    g = ou_terminal_loss(lnpi=ref_process.target,
                         sigma=ref_process.sigma,
                         tfinal=ref_process.tfinal,
                         brown=False)

    loss = relative_kl_objective(augmented_trajectory=aug_trajectory,
                                 g=g,
                                 stl=False,
                                 trim=2,
                                 dim=ref_process.data_dim)

    print(loss)
    adam = torch.optim.Adam(ref_process.drift_network.parameters(), lr=0.0005)
    adam.zero_grad()
    loss.backward()

    # for name, param in ref_process.drift_network.named_parameters():
    #     if param.requires_grad:
    #         print(name, param.grad)
    #
    # torch.nn.utils.clip_grad_norm_(ref_process.drift_network.parameters(), max_norm=1)



    adam.step()

    e_t = time.time()

    print(f"Epoch {i}, time of execution: {e_t - s_t}")

    # if i % 10 == 0:
    #     x, v = [], []
    #     for i in aug_trajectory[0][:, -1, :10]:
    #         v.append(i[0].detach().numpy())
    #         x.append(i[1].detach().numpy())
    #
    #     fig, axes = plt.subplots(1, 2, constrained_layout=True)
    #     ax = axes[0]
    #     ax.scatter(x, v, marker=".", alpha=0.05, rasterized=True)
    #     ax.set_xlim(-5, 5)
    #     ax.set_ylim(-5, 5)
    #     ax.set_xlabel("$x_0$")
    #     ax.set_ylabel("$v$")
    #
    #     ax = axes[1]
    #     ax.scatter(x, v, marker=".", alpha=0.05, rasterized=True)
    #     ax.set_xlim(-20, 20)
    #     ax.set_ylim(-9, 9)
    #     ax.set_xlabel("$x_0$")
    #     ax.set_ylabel("$v$")
    #
    #     plt.show()


tensor(105.3028, grad_fn=<MeanBackward0>)
Epoch 1, time of execution: 35.748926401138306
tensor(88.7584, grad_fn=<MeanBackward0>)
Epoch 2, time of execution: 37.634530782699585
tensor(66.5441, grad_fn=<MeanBackward0>)
Epoch 3, time of execution: 38.144811153411865
tensor(64.9721, grad_fn=<MeanBackward0>)
Epoch 4, time of execution: 38.173665046691895
tensor(44.4605, grad_fn=<MeanBackward0>)
Epoch 5, time of execution: 38.6832230091095
tensor(52.9727, grad_fn=<MeanBackward0>)
Epoch 6, time of execution: 38.69487166404724
tensor(49.5019, grad_fn=<MeanBackward0>)
Epoch 7, time of execution: 39.07055687904358
tensor(46.6311, grad_fn=<MeanBackward0>)
Epoch 8, time of execution: 38.78816103935242
tensor(41.4237, grad_fn=<MeanBackward0>)
Epoch 9, time of execution: 39.06094408035278
tensor(39.1497, grad_fn=<MeanBackward0>)
Epoch 10, time of execution: 38.50442314147949
tensor(36.7315, grad_fn=<MeanBackward0>)
Epoch 11, time of execution: 38.90545201301575
tensor(22.2586, grad_fn=<MeanBackward

In [4]:
#aug_trajectory_for_plotting = model_def(batch_size=300, is_training=True, ode=False)

In [5]:
#print(aug_trajectory_for_plotting[0])#.shape)

In [6]:
# v = []
# for traj in all_trajectories:
#     curr = []
#     for i in traj[0][:, -1, :10]:
#         curr.append(i[0].detach().numpy())
#     v.append(np.mean(curr))
#
# print(v)
#
# for i in carrillo(torch.tensor(v)):
#     print(round(float(i), 3))

In [7]:
aug_trajectory_for_plotting = model_def(batch_size=200, is_training=True, ode=False)

In [ ]:
aug_trajectory_for_plotting_ode = model_def(batch_size=10, is_training=False, ode=True)
traj = aug_trajectory_for_plotting_ode[0]
for i in range(99):
    t_i = model_def(batch_size=10, is_training=True, ode=True)
    traj = torch.concat([traj, t_i[0]])
    print(i)

# for loop it

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48


In [ ]:
print(torch.concat([aug_trajectory_for_plotting_ode[0], aug_trajectory_for_plotting_ode1[0]]).shape)

In [ ]:

sde_targ = aug_trajectory_for_plotting[0][:, -1,:2].detach().numpy()
plt.plot(sde_targ[:, 0], sde_targ[:, 1], ".", alpha=0.4)

In [ ]:
ode_targ = aug_trajectory_for_plotting_ode[0][:, -1,:2].detach().numpy()
ode_targ = traj[:, -1,:2].detach().numpy()
plt.plot(ode_targ[:, 0], ode_targ[:, 1], ".", alpha=0.4)

In [ ]:
data = []
for sample in aug_trajectory_for_plotting_ode[0]:
    print(sample[-1])
    s = sample[-1][0].detach().numpy()
    data.append(s)
    print(s)


In [ ]:
# generate some sample data
#data = np.random.randn(1000)
# create the histogram
plt.hist(data, bins=50)

# set the x and y axis labels
plt.xlabel('Value')
plt.ylabel('Frequency')

# show the plot
plt.show()

In [ ]:
# plot data
fig, axes = plt.subplots(1, 2, constrained_layout=True)
ax = axes[0]
ax.scatter(x, v, marker=".", alpha=0.05, rasterized=True)
ax.set_xlim(-5, 5)
ax.set_ylim(-5, 5)
ax.set_xlabel("$x_0$")
ax.set_ylabel("$v$")

# plot analytic log-likelihood
ax = axes[1]
ax.scatter(x_i, v_i, marker=".", alpha=0.05, rasterized=True)
ax.set_xlim(-20, 20)
ax.set_ylim(-9, 9)
ax.set_xlabel("$x_0$")
ax.set_ylabel("$v$")

plt.show()

In [ ]:
x, v = [], []
for i in aug_trajectory_for_plotting[0][:, -1, :10]:
    v.append(i[0].detach().numpy())
    x.append(i[1].detach().numpy())

fig, axes = plt.subplots(1, 2, constrained_layout=True)
ax = axes[0]
ax.scatter(x, v, marker=".", alpha=0.05, rasterized=True)
ax.set_xlim(-5, 5)
ax.set_ylim(-5, 5)
ax.set_xlabel("$x_0$")
ax.set_ylabel("$v$")

ax = axes[1]
ax.scatter(x, v, marker=".", alpha=0.05, rasterized=True)
ax.set_xlim(-20, 20)
ax.set_ylim(-9, 9)
ax.set_xlabel("$x_0$")
ax.set_ylabel("$v$")

plt.show()

In [ ]:

t1, sam1 = funnel(10, 1)
t3, sam3 = funnel(10, 3)

b1 = sam1(100)
b3 = sam3(100)

print(sum(t1(b1)), sum(t3(b1)))

print(sum(t1(b1)), sum(t3(b3)))